In [1]:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from common.utils import get_data_urls, fetch_data, get_labeled_files

In [4]:
#data_urls = get_data_urls()
#fetch_data(data_urls)

data_files, class_labels = get_labeled_files()

In [5]:
from sklearn.model_selection import train_test_split

train_files, test_files, y_train, y_test = train_test_split(data_files, class_labels,
                                                            test_size=0.2, random_state=44)
print(f"Training set size is\t {len(train_files)}")
print(f"Test set size is\t {len(test_files)}")

Training set size is	 7479
Test set size is	 1870


In [7]:
from mailparser import mailparser
from common.message import Message

messages = []
for i, message in enumerate(train_files[:100]):
    messages.append(Message(mailparser.parse_from_file(message), y_train[i]))

More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from unknown HELO mfrenchw2k mfrench42@62.254.163.42 with login by smtp.mail.vip.sc5.yahoo.com with SMTP; 13 Aug 2002 09:18:55 -0000
More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from unknown HELO mfrenchw2k mfrench42@62.254.163.42 with login by smtp.mail.vip.sc5.yahoo.com with SMTP; 13 Aug 2002 09:18:55 -0000
More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from chekitb5876.com 195.27.92.154 by asptown.co.kr 211.52.47.8 with Nmail V3.1 20010905 S for <jm@netnoteinc.com> from <jimmiester@hanmesoft.co.kr>; Sun, 02 Jun 2002 02:37:30 +0900
More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*

In [9]:
from common.database_populator import DatabasePopulator

db_populator = DatabasePopulator("../db/spam.db")

In [10]:
db_populator.populate_message_table(messages)

In [17]:
from common.message import Message
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

class MessageTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, input="filename"):
        '''string {‘filename’, ‘file’, ‘content’}, '''
        self.input = input
        self.messages = []
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        '''X is a list of files'''
        feature_aggregator = []
        if isinstance(X, str):
             raise ValueError("Must be a list or iterable, not a string")
        for x in X:
            if self.input == "content":
                mailparser_obj = mailparser.parse_from_string(x)
            elif self.input == "file":
                mailparser_obj = mailparser.parse_from_file_obj(x)
            elif self.input == "filename":
                print(x)
                mailparser_obj = mailparser.parse_from_file(x)
            
            feature_aggregator.append(Message(mailparser_obj).extract_features())
            
        return pd.DataFrame.from_records(feature_aggregator)


pipeline = Pipeline([
    ("message_trf", MessageTransformer()),
    ("vectorizer", 
         ColumnTransformer([
            ("tdidf_subj_vectorizer", TfidfVectorizer(max_features=200), "subject_tokens"),
            ("tdidf_body_vectorizer", TfidfVectorizer(max_features=1000), "body_tokens"),
         ])
    )
])

In [5]:
import mailparser

In [6]:
mailparser.parse_from_file(r"data/spam/0369.2530542de47d461ccb925fcafc6f0ad5")

OSError: [Errno 22] Invalid argument

In [54]:
test = pipeline.fit_transform(train_files[:100])

data\easy_ham\1444.9c769de9af58bfba4b3e6b0ca034f2b8
data\easy_ham\01122.69d3ebe6675d828f75af6ca1f905802c
data\spam_2\00887.7a8cc64a563c42af29184459e6c1a97f
data\easy_ham\1161.9bcd69bccfeb05378e3e36fa62b16f7d
data\easy_ham_2\00990.3a91d53e9908fc65cf31a6c57e2a844d
data\easy_ham\1535.92d5d08b41907f13b4553115c772506b
data\easy_ham\01942.30343e06573f363493a13a5a2b4ebc6f
data\easy_ham\0782.03d024d74f36ef492a9d8d5de300fb94
data\easy_ham_2\00346.2f9b84c3f47ce85fcf3f7ef74b9e0ac6
data\easy_ham_2\00481.5afc99aa0e2940b2f61c1a1bf9ea0b49
data\hard_ham\00065.84b8fba96e680358bbd2c961fe356982
data\easy_ham\1255.471b16ac442ca5fd6e94eecf29582676
data\easy_ham\1609.d3849c49c9377d824b5ab007f36e96f8
data\spam\0369.2530542de47d461ccb925fcafc6f0ad5


OSError: [Errno 22] Invalid argument

In [ ]:
test

In [81]:
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB

xgb_clf = xgb.XGBClassifier().fit(test, y_train[:400])
naive_clf = MultinomialNB().fit(test, y_train[:400])

In [82]:
import numpy as np

from sklearn.model_selection import cross_val_score

xgb_train_accuracy = np.mean(cross_val_score(xgb_clf, test, y_train[:400], cv=5))
naive_train_accuracy = np.mean(cross_val_score(naive_clf, test, y_train[:400], cv=5))

In [85]:
len(train_files)

7483

In [83]:
print("XGB Accuracy:\t", xgb_train_accuracy)
print("Naive Bayes Accuracy:\t", naive_train_accuracy)

XGB Accuracy:	 0.9199999999999999
Naive Bayes Accuracy:	 0.9349999999999999


In [723]:
import joblib

joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']